# Semester Project:
# exploring Differentiable Neural Computers and applications

## Introduction
What we want to present in the following notebook is a cutting-edge topic in machine learning, related to a particular model of Neural Network called **Differentable Neural Computer** (DNC) which puts together a Neural Network and an external memory from which the Netwoek can read and write. The main source of this noteboook is based on the article and paper which can be found [here](https://deepmind.com/blog/differentiable-neural-computers/).
<br>

We start with a presentation of the DNC and how they are enhanced form *Neural Turing Machines*. Then we move to the core topic of this networks which are the three **differentiable attention mechanisms** with whihc the network performs the read and writes operations. We propose then a code for the implementation of these Networks, testing their performances and limitations. Finally we will try to implement the graph task presented in the paper and adapt it to a different problem.
<hr>

## DNC
The Differentiable Neural Computers are neural networks with a coupled memory, where the dimension of this memory does not affect the behaviour of the network. The memory can be tought as a RAM and the Network as a controller which is differentiable using gradient descent-like tecniques.
<br>
Comparing this Network with the respect of other whihc present memory states such as RNN or LSTM, DNCs do not store information whihc is tigth coupled with the model itself. Instead, they have the possibility to selectively read and write to and from memory locations, creating a separation between state preservation and content (data).
<br>

In the litterature, another network capable of read and write using an external memory came out in the past, called **Neural Turing Machine** (NTM). Altought the concept is similar to the DNC, with a Network acting as controller and an external memory, they are more restricted in the method with which the Neural Network access the memory. 

<br>
![DNC](./figures/DNC_architecture.png)
<br>

The image shows the architecture of a DNC Neural Network. In this specif example, the controller is a Recursive Neural Network (a LSTM is also possible) which is receiving an input (from a dataset) and the previous step readings from the memory. The output of the network is made of two parts:
- The output of the Neural Network, which constitute our target
- A vector called **'Interface Vector'** which is used by the read and write heads to interact with the memory

As it's clear by now, the DNC is mainly composed of two parts interacting together: the **Neural Netwoerk** itself (**a**) and the **Heads** (**b**) which performs the operations over the **Memory** (**c**). Those oprations are the key differences of DNC over NTM and will be described in details in the next paragraph. 

The Heads are particular components which are dealing with the content of the memory. It's possible to define as many read and write heads as needed, all of them will receive weights vector which are used to define the location over which perform the read and write operations. The differentiability of these parts allows the network to learn how to perform these operations simple by looking at the error during the trainign process and adjusting the weights.

Finally, in addition to the Memory there is an additional information whcih is saved and used during the read and writes operations. Those are the **Memory Usage and Temporal Links** (**d**) which is an imprvement over the NTM. Those links and usage information allows to dynamically allocate the content during the reads and to have notion of temporal assosciation between entries of the memory. That is, it's possibile to know the sequence of the writtings which is extremely important when the Network has to deal to sequential tasks such as graph path search.
<hr>


## Memory Interactions

### Overview
The core mechanism of DNCs is the possibility to write and read from an external memory matrix. As mentioned, the difference with the NTMs is in defining more **attention mechanisms**. This is different from the address mechanisms in conventional computers, where there is a mapping between the address and the content of the memory. The idea here is to define the weights over the location of the Memory. Those weights represents a degree (we can immagine it has a filter) that indicates how much the locations in the Memory matrix are involved during read and writing operations.
<br>
Briefly:
- a read vector $\mathbf{r}$ is returned after a read operation which involves a read weights matrix $\mathbf{w^r}$:<br>
    $\mathbf{r}=\sum_{i=1}^N M[i,j]w^r[i]$ for $j=1, \dots,W$
    
- an erase vector $\mathbf{e}$ is applied using the write weights matrix $\mathbf{w^w}$, then a a write vector is added $\mathbf{v}$:<br>
    $\mathbf{v}: M[i,j] \leftarrow M[i,j](1-\mathbf{w^w}[i]\mathbf{e}[j]+\mathbf{w^w}[i]\mathbf{v}[j])$

The units which determines this operations are the read and write Heads. 

### Differentiable Attention
There are three forms of differentiable attention. This is more a fancy terminology to call the three pricipal techniques involved in in order to address the memory. Before going into the details we want to recall some of the techniques which are instead used in NTM to appreasciate the difference with the DNC. 

There are two mechanism for addressing the memory and the Neural Tuning Machines combines both:
- *content-based addressing*: focuses attention on locations based on the similarity between their current values and values emitted by the controller. This is related to the content addressing of the Hopfield networks, the controller needs to generate a value which is an approximation of the one stored to then retrieve the exact location. 
- *location-based addressing*: it's the traditional approach to address the memory. For arithmetic operation where we need to define variables the conent based in not enough, we need the location of the variable to perform the operation.

Differentiable Neural Computers uses a *content-based addressing* paired with other two techniques which allows for a **Dynamic Memory Addressing** counteracting the major drawbacks of the NTM:
1. NTMs do not avoid possible overlapping and interfere among blocks of allocating memory. DNCs instead overcame this problem due to there is only a single free at each write time.
2. NTMs does not allow for freeing location of memory which are not used and this can be a problem when processing long sequences. DNCs instead can free memory locations based on the usage weights.
3. NTMs sequential information is preserved only if the content is written in consecutive locations. DNCs uses an additional temporal link matrix avoidin the restriction to continuous locations only. 

For now we explain the concepts of these three differentiable attention. A more detailed analysis will be given in the following paragraphs

#### Content base addressing
This form is used to determine the similarity measure between the vector emmited by the controller and the content of the memory. The measure is a cosine similarity function that returns weights which are then used by the read head for associative recall or by the write head to modify the conetent of the memory. In addition, if the key only match a part of the conentent of the memory this is still useful and can lead to the retrieval of that location. This may be due to the key may not have all the information which instead are stored in the memory.

#### Temporary links
This form keep track of the transitions betweens locations which were consecutively written using an $LxL$ temporal *link matrix* $L$. This matrix associate a weight from 0 to 1 for each pair of locations in the matrix, where the entry $L[i,j]$ is the temporal relation between the location $i$ and $j$. The weight is closer to $1$ if the locations $i$ was written after $j$, otherwise the value is closer to $0$. This gives to the Neural Network the ability to recover sequences following the order under which they were written.
<br>

The product $L\mathbf{w}$ creat a smoothing effect, shifting the focus forwards the locations after those emphasized by $\mathbf{w}$. That is, after a writting which is based on $\mathbf{w}$

#### Usage 
This form is used to allocate the memory for writting. The usage is a value between $0$ and $1$, with a weighting to select unsued locations that is delivered to the writing head. The usage is incremented after each write to that location and decreased after each read. The good property is that this is independent from the memory size and content. This allows the network to be trained and then upgrade to a larger one.